Imports

In [ ]:
# Import necessari
import cv2
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import json
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
#controllo il massimo valore di altezza e larghezza per fare il padding
import cv2
import os

# Directory containing the images
data_dir = 'Resizedmasks'

# Initialize variables to store max height and width
max_height = 0
max_width = 0

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape
        if(height != 1000 or width != 1000):
            print(filename)
            print(image.shape)

        # Update max_height and max_width if necessary
        if height > max_height:
            max_height = height
        if width > max_width:
            max_width = width

        

#Image with different max_height and max_width: Queretaro-TEC_parcheggi.png
print("Maximum Height:", max_height)
print("Maximum Width:", max_width)

Image resize

In [ ]:
import cv2
import os

# Directory containing the images
data_dir = 'Dataset'
# Directory to save resized images
resized_dir = 'ResizedDataset'

# Ensure the resized_dir exists
os.makedirs(resized_dir, exist_ok=True)

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape

        # If the image is larger than 1000 in either dimension, resize it
        if height > 1000 or width > 1000:
            # Calculate the scale factor
            scale_factor = 0.5  # Reduce size by half

            # Calculate the new dimensions of the image
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)

            # Resize the image
            image = cv2.resize(image, (new_width, new_height))

        # Save the image (resized or original) to the new directory
        new_image_path = os.path.join(resized_dir, filename)
        cv2.imwrite(new_image_path, image)
        

Mask resize and padding

In [ ]:
import cv2
import os

# Directory containing the images
data_dir = 'image_newmask'
# Directory to save resized images
resized_dir = 'Resizedmasks'

# Ensure the resized_dir exists
os.makedirs(resized_dir, exist_ok=True)

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape

        # If the image is larger than 1000 in either dimension, resize it
        if height > 1000 or width > 1000:
            # Calculate the scale factor
            scale_factor = 0.5  # Reduce size by half

            # Calculate the new dimensions of the image
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)

            # Resize the image
            image = cv2.resize(image, (new_width, new_height))

        ## MASK PADDING
        max_height = 1056
        max_width = 1056
        padding_height = max(0, max_height - image.shape[0])
        padding_width = max(0, max_width - image.shape[1])
        mask_padded = cv2.copyMakeBorder(image, 0, padding_height, 0, padding_width, cv2.BORDER_CONSTANT, value=0)
        # Save the image (resized or original) to the new directory
        new_image_path = os.path.join(resized_dir, filename)
        cv2.imwrite(new_image_path, mask_padded)


In [ ]:
for filename in os.listdir('Dataset_edge'):
    image_path = os.path.join('Dataset_edge', filename)
    image = cv2.imread(image_path)
    if image.shape[0] != 1056 or image.shape[1] != 1056:
        print(image_path)
        print(image.shape[0],image.shape[1])

Store canny edge

In [ ]:
dataset_path = 'ResizedDataset'
edge_path = 'Dataset_edge'

os.makedirs(edge_path, exist_ok=True)

def get_edge_image(image, low_threshold, high_threshold):
    # Convert the image to grayscale if it's not already
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Canny edge detection
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    
    return edges
#edge detection + padding immagini
for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        
        input_image_path = os.path.join(dataset_path, filename)
        image = cv2.imread(input_image_path)
        # Ottieni l'immagine del bordo
        edge_image = get_edge_image(image, low_threshold=200, high_threshold=300)
        
        # Apply padding to the edge image
        max_height = 1056
        max_width = 1056
        padding_height = max(0, max_height - edge_image.shape[0])
        padding_width = max(0, max_width - edge_image.shape[1])
        edge_image_padded = cv2.copyMakeBorder(edge_image, 0, padding_height, 0, padding_width, cv2.BORDER_CONSTANT, value=0)
        #Verificare se il nome del file contiene "_testing" o "_training"
        if "_testing" in filename or "_training" in filename:
            # Rinominare il file rimuovendo "_testing" o "_training" dal nome
            new_filename = filename.replace("_testing", "").replace("_training", "")
            
            print(f"Rinominato il file {filename} in {new_filename}")
            # Salva l'immagine del bordo nella cartella di output
            output_image_path = os.path.join(edge_path, new_filename)
            cv2.imwrite(output_image_path, edge_image_padded)

print("Il rilevamento dei bordi è stato applicato a tutte le immagini nella cartella.")


In [ ]:
import os
import shutil

# Define the paths to the mask and image folders
mask_folder = 'Dataset_mask'
image_folder = 'Dataset_edge'

# Create a new folder to store the combined images and masks
combined_folder = 'Dataset_combined'
os.makedirs(combined_folder, exist_ok=True)

# Copy all files from the mask folder to the combined folder
for mask_file in os.listdir(mask_folder):
    shutil.copy2(os.path.join(mask_folder, mask_file), combined_folder)

# Copy all files from the image folder to the combined folder
for image_file in os.listdir(image_folder):
    shutil.copy2(os.path.join(image_folder, image_file), combined_folder)

print("Masks and images have been combined and stored in the 'Combined' folder.")

Dataset preparation

DATASET CLASS

In [ ]:
import os
import glob
from PIL import Image
import os
import glob
from PIL import Image

class ParkingLotDataset(Dataset):
    def __init__(self, root_img, root_msk, pairs=None, transforms=None):
        self.root_img = root_img
        self.root_msk = root_msk
        self.transforms = transforms

        if pairs is None:
            # Get all image files
            self.image_paths = sorted(glob.glob(os.path.join(root_img, '*.png')))

            # Get all mask files
            self.mask_paths = sorted(glob.glob(os.path.join(root_msk, '*.png')))

            # Pair image and mask files based on their filenames
            #self.pairs = [(image_path, mask_path) for image_path in self.image_paths for mask_path in self.mask_paths if os.path.splitext(os.path.basename(image_path))[0] == os.path.splitext(os.path.basename(mask_path))[0]]
            self.pairs = []

            for image_path in self.image_paths:
                image_filename = os.path.splitext(os.path.basename(image_path))[0]
                mask_filename = f"{image_filename}_SegmentationClass.png"
                mask_path = os.path.join(root_msk, mask_filename)
                if os.path.exists(mask_path):
                    self.pairs.append((image_path, mask_path))

        else:
            self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        image_path, mask_path = self.pairs[idx]

        # Load image
        image = Image.open(image_path)
        image_array = np.array(image)

        # Apply transformations
        if self.transforms:
            image_array = self.transforms(image_array)
            
        
        #mask = Image.open(mask_path)
        #mask_array = np.array(mask)
        mask_array = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Apply transformations
        if self.transforms:
            mask_array = self.transforms(mask_array)
            


        return image_array, mask_array
        


DATASET SLPIT

In [ ]:
import random
from torch.utils.data import random_split
from torchvision.transforms import functional as F

# magari rifare il dataloader con due cartelle
image_path = '/kaggle/input/dataset-conedge/Dataset_edge'
mask_path = '/kaggle/input/dataset-conedge/Resizedmasks'


transform = transforms.Compose([
    
    transforms.ToTensor(),
    # Add other transforms here as needed
])


# Create the dataset
dataset = ParkingLotDataset(image_path, mask_path, transforms=transform)

# Define the proportions for the split
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

total_size = len(dataset)
print(total_size)

# Shuffle the dataset
random.shuffle(dataset.pairs)

# Calculate the sizes of each split
total_size = len(dataset)
print(total_size)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size

# Split the dataset
train_pairs = dataset.pairs[:train_size]
val_pairs = dataset.pairs[train_size:train_size+val_size]
test_pairs = dataset.pairs[train_size+val_size:]

# Create datasets for each split
train_dataset = ParkingLotDataset(image_path, mask_path, pairs=train_pairs, transforms=transform)
val_dataset = ParkingLotDataset(image_path, mask_path, pairs=val_pairs, transforms=transform)
test_dataset = ParkingLotDataset(image_path, mask_path, pairs=test_pairs, transforms=transform )

# Now you can create data loaders for each split
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False) 


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a DataLoader object called train_loader
num_samples = 10

# Iterate over batches from the DataLoader
for batch_idx, (images, masks) in enumerate(train_loader):
    # Get the number of samples in the current batch
    batch_size = images.size(0)

    # Iterate over the samples in the batch
    for i in range(batch_size):
        # Convert the image tensor to a NumPy array
        image_array = images[i].permute(1, 2, 0).numpy()

        # Convert the mask tensor to a NumPy array
        mask_array = masks[i].permute(1, 2, 0).numpy()

        # Plot the image and the combined mask
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(image_array, cmap='gray')
        plt.title(f'Image {batch_idx * batch_size + i + 1}')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(image_array, cmap='gray')
        plt.imshow(mask_array, cmap='gray', alpha=0.5)
        plt.title(f'Image {batch_idx * batch_size + i + 1} with Masks')
        plt.axis('off')

        plt.show()

        # Break after the first 10 samples
        if batch_idx * batch_size + i == num_samples - 1:
            break

    # Break after processing one batch
    if batch_idx == 0:
        break

Model

In [1]:
import torch
from torchvision import models
from torch import nn

class SmallUNet(nn.Module):
    def __init__(self):
        super(SmallUNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)

        self.upconv2 = nn.Conv2d(128, 32, 3, padding=1)
        self.upconv1 = nn.Conv2d(64, 32, 3, padding=1)

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.final_conv = nn.Conv2d(32, 1, 1)

    def forward(self, x):
        conv1 = F.relu(self.conv1(x))
        x = self.maxpool(conv1)

        conv2 = F.relu(self.conv2(x))
        x = self.maxpool(conv2)

        x = F.relu(self.conv3(x))

        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)
        x = F.relu(self.upconv2(x))

        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)
        x = F.relu(self.upconv1(x))

        out = torch.sigmoid(self.final_conv(x))

        return out

Train

In [ ]:
from torch.utils.data import DataLoader
from torch import optim
import torch.nn.functional as F
import wandb

# Start a new run
#run = wandb.init(project='Parking_lot_zones', entity='Fede_occe')




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SmallUNet().to(device)
optimizer = optim.sgd(model.parameters(), lr=0.01)
#wandb.watch(model)
epochs = 5

for epoch in range(epochs):
    model.train()
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = F.binary_cross_entropy_with_logits(outputs, masks)
        loss.backward()
        optimizer.step()

        #wandb.log({"Train Loss": loss.item()})

    model.eval()
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)
            val_loss = F.binary_cross_entropy_with_logits(outputs, masks)

            #wandb.log({"Validation Loss": val_loss.item()})

    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")